Train the model

In [6]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import joblib

In [7]:
def train_graph_model(): 
    
    df = pd.read_csv("Recommendation_graph_data.csv")
    X = df[["num_num", "num_cat", "has_dt"]] 
    y = df.drop(columns=["num_num", "num_cat", "has_dt"]) 
    
    
    model = MultiOutputClassifier(
        RandomForestClassifier(n_estimators=100, random_state=42) 
    )
    
    model.fit(X, y)
    
   
    joblib.dump(model, "graph_recommender_model.pkl")
    print("Model trained and saved!")

if __name__ == "__main__":


    train_graph_model()

Model trained and saved!


 Column Analysis

In [8]:
def analyze_columns(df):
    num_num = 0
    num_cat = 0
    has_dt = 0 
    
    for col in df.columns: 
        dtype = df[col].dtype
        
        if pd.api.types.is_numeric_dtype(dtype): 
            num_num += 1
        else:
           
            temp_col = pd.to_datetime(df[col], errors="coerce")

            if temp_col.notna().sum() > 0: 
                has_dt = 1
            else:
                num_cat += 1

    return num_num, num_cat, has_dt

Get user columns

In [9]:
def get_user_columns(dataset_path):
    df = pd.read_csv(dataset_path)
    print("\nAvailable columns:", df.columns.tolist())
    
    while True:
        selected = input("Enter columns (comma-separated): ").strip().split(',')
        selected = [col.strip() for col in selected]
        valid_cols = [col for col in selected if col in df.columns]
        
        if valid_cols:
            return valid_cols
        print("Error: No valid columns selected. Try again.") 

In [ ]:
Main function

In [ ]:
def main():
    
    model = joblib.load("graph_recommender_model.pkl")
    
  
    dataset_path = "Datasets/titanic.csv" 
    selected_cols = get_user_columns(dataset_path)
    
    df = pd.read_csv(dataset_path)[selected_cols]
    num_num, num_cat, has_dt = analyze_columns(df)
    
    input_df = pd.DataFrame(
        [[num_num, num_cat, has_dt]],
        columns=["num_num", "num_cat", "has_dt"]  
    )
    
    
    pred = model.predict(input_df)
    
    graphs = ["Bar", "Line", "Scatter", "Area", "Histogram", "Box", 
             "Funnel", "Tree", "Pie"]
    recommendations = [g for g, p in zip(graphs, pred[0]) if p == 1]
    
    print("\nRecommended Graph Types:")
    print("- " + "\n- ".join(recommendations) if recommendations else "No suitable graphs found")

if __name__ == "__main__":
    main()


Available columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
